In [1]:
import sys
sys.path.append("..")

In [2]:
from collections import Counter, namedtuple

from resources.utils import get_puzzle_input

### Part 1


"Situation critical," the device announces. "Destination indeterminate. Chronal interference detected. Please specify new target coordinates."

The device then produces a list of coordinates (your puzzle input). Are they places it thinks are safe or dangerous? It recommends you check manual page 729. The Elves did not give you a manual.

If they're dangerous, maybe you can minimize the danger by finding the coordinate that gives the largest distance from the other points.

Using only the Manhattan distance, determine the area around each coordinate by counting the number of integer X,Y locations that are closest to that coordinate (and aren't tied in distance to any other coordinate).

Your goal is to find the size of the largest area that isn't infinite. For example, consider the following list of coordinates:

```
1, 1
1, 6
8, 3
3, 4
5, 5
8, 9
```

If we name these coordinates A through F, we can draw them on a grid, putting 0,0 at the top left:

```
..........
.A........
..........
........C.
...D......
.....E....
.B........
..........
..........
........F.
```

This view is partial - the actual grid extends infinitely in all directions. Using the Manhattan distance, each location's closest coordinate can be determined, shown here in lowercase:

```
aaaaa.cccc
aAaaa.cccc
aaaddecccc
aadddeccCc
..dDdeeccc
bb.deEeecc
bBb.eeee..
bbb.eeefff
bbb.eeffff
bbb.ffffFf
```

Locations shown as . are equally far from two or more coordinates, and so they don't count as being closest to any.

In this example, the areas of coordinates A, B, C, and F are infinite - while not shown here, their areas extend forever outside the visible grid. However, the areas of coordinates D and E are finite: D is closest to 9 locations, and E is closest to 17 (both including the coordinate's location itself). Therefore, in this example, the size of the largest area is 17.

What is the size of the largest area that isn't infinite?

In [3]:
Point = namedtuple('Point', ['x', 'y'])

def manhatten_distance(left, right):
    return abs(left.x - right.x) + abs(left.y - right.y)

In [4]:
assert manhatten_distance(Point(1, 1), Point(3, 3)) == 4
assert manhatten_distance(Point(3, 3), Point(1, 1)) == 4

In [5]:
def nearest_point(point, locations):
    nearest_point = None
    min_distance = None
    equidistant = False
    for x in range(len(locations)):
        distance = manhatten_distance(point, locations[x])
        if min_distance is None or distance < min_distance:
            nearest_point = x
            min_distance = distance
            equidistant = False
        elif min_distance == distance:
            equidistant = True

    if equidistant:
        return None
    else:
        return nearest_point           

In [6]:
test_points = [
    Point(1, 1),
    Point(1, 6),
    Point(8, 3),
    Point(3, 4),
    Point(5, 5),
    Point(8, 9),
]

In [7]:
assert nearest_point(Point(1, 1), test_points) == 0
assert nearest_point(Point(1, 3), test_points) == 0
assert nearest_point(Point(1, 4), test_points) is None
assert nearest_point(Point(2, 4), test_points) == 3
assert nearest_point(Point(1, 5), test_points) == 1
assert nearest_point(Point(5, 7), test_points) == 4

In [8]:
def south_east_corner(locations):
    max_x = None
    max_y = None
    
    for point in locations:
        if max_x is None or point.x > max_x:
            max_x = point.x
        if max_y is None or point.y > max_y:
            max_y = point.y
    
    return Point(max_x, max_y)           

In [9]:
assert south_east_corner(test_points) == Point(8, 9)

In [10]:
# Only need to consider points in the up to max x, y
# Then exclude areas that touch the right and bottom

class NearestGrid:
    def __init__(self, locations):
        self.locations = locations
        self.se_corner = south_east_corner(locations)
        self._build_grid()
    
    def _build_grid(self):
        max_x, max_y = self.se_corner
        
        rows = []
        
        for y in range(max_y + 1):
            columns = []
            for x in range (max_x + 1):
                point = Point(x, y)
                columns.append(nearest_point(point, self.locations))
            rows.append(columns)
        
        self.grid = rows
     
    @property
    def unbounded_areas(self):
        unbounded = set()
        for row in self.grid:
            east = row[-1]
            west = row[0]
            if east is not None:
                unbounded.add(east)
            if west is not None:
                unbounded.add(west)

        for north in self.grid[0]:
            if north is not None:
                unbounded.add(north)

        for south in self.grid[-1]:
            if south is not None:
                unbounded.add(south)
        
        return unbounded
    
    @property
    def areas(self):
        area_ctr = Counter()
        
        for row in self.grid:
            for col in row:
                if col is not None:
                    area_ctr[col] += 1
        
        return area_ctr 
    
    def max_bounded(self):
        unbounded = self.unbounded_areas
        max_area = None
        
        for n, v in self.areas.items():
            if n in unbounded:
                continue
            if max_area is None or v > max_area:
                max_area = v
        
        return max_area
            

    def __str__(self):
        row_output = []
        for row in self.grid:
            col_output = []
            for col in row:
                if col is None:
                    col_output.append('.')
                else:
                    col_output.append(chr(65 + col))
            row_output.append(''.join(col_output))

        return '\n'.join(row_output)


In [11]:
grid = NearestGrid(test_points)
print(grid)

AAAAA.CCC
AAAAA.CCC
AAADDECCC
AADDDECCC
..DDDEECC
BB.DEEEEC
BBB.EEEE.
BBB.EEEFF
BBB.EEFFF
BBB.FFFFF


In [12]:
grid.unbounded_areas

{0, 1, 2, 5}

In [13]:
grid.max_bounded()

17

In [14]:
puzzle_input = get_puzzle_input('/tmp/day_6.txt')

In [15]:
puzzle_points = []
for p in puzzle_input:
    x, y = p.split(', ')
    puzzle_points.append(Point(int(x), int(y)))

In [16]:
grid = NearestGrid(puzzle_points)
grid.max_bounded()

3907

### Part 2

On the other hand, if the coordinates are safe, maybe the best you can do is try to find a region near as many coordinates as possible.

For example, suppose you want the sum of the Manhattan distance to all of the coordinates to be less than 32. For each location, add up the distances to all of the given coordinates; if the total of those distances is less than 32, that location is within the desired region. Using the same coordinates as above, the resulting region looks like this:

```
..........
.A........
..........
...###..C.
..#D###...
..###E#...
.B.###....
..........
..........
........F.
```

In particular, consider the highlighted location 4,3 located at the top middle of the region. Its calculation is as follows, where abs() is the absolute value function:

```
Distance to coordinate A: abs(4-1) + abs(3-1) =  5
Distance to coordinate B: abs(4-1) + abs(3-6) =  6
Distance to coordinate C: abs(4-8) + abs(3-3) =  4
Distance to coordinate D: abs(4-3) + abs(3-4) =  2
Distance to coordinate E: abs(4-5) + abs(3-5) =  3
Distance to coordinate F: abs(4-8) + abs(3-9) = 10
Total distance: 5 + 6 + 4 + 2 + 3 + 10 = 30
```

Because the total distance to all coordinates (30) is less than 32, the location is within the region.

This region, which also includes coordinates D and E, has a total size of 16.

Your actual region will need to be much larger than this example, though, instead including all locations with a total distance of less than 10000.

What is the size of the region containing all locations which have a total distance to all given coordinates of less than 10000?

In [17]:
def total_distance(point, locations):
    return sum(manhatten_distance(point, location) for location in locations)

In [18]:
assert total_distance(Point(4, 3), test_points) == 30

In [19]:
def num_in_dist(points, max_dist):
    max_x, max_y = south_east_corner(points)
    
    # Need to consider extra points off the N,E,S and W of
    # the grid. Given the min manhatten distance to any point
    # is the extra length the min possible distance to all points
    # is max_dist / num of points
    offset = max_dist // len(points)
    
    area = 0
    for y in range(-offset, max_y + offset):
        for x in range(-offset, max_x + offset):
            point = Point(x, y)
            
            if total_distance(point, points) < max_dist:
                area += 1
    
    return area

In [20]:
num_in_dist(test_points, 32)

16

In [21]:
num_in_dist(puzzle_points, 10000)

42036